In [ ]:
#pip install openai

In [ ]:
#test
import os
from openai import OpenAI

client = OpenAI(api_key=api_key)

response = client.responses.create(
    model="gpt-4o",
    instructions="You are a coding assistant that talks like a pirate.",
    input="How do I check if a Python object is an instance of a class?",
)

print(response.output_text)

In [38]:
import pandas as pd

df = pd.read_csv("yishan_answers.csv")

# Combine past answers into a single string
yishan_knowledge = "\n".join([f"Q: {row['Question']}\nA: {row['Answer']}" for _, row in df.iterrows()])
# print(yishan_knowledge)

In [ ]:
df_test = pd.read_csv('yishan_test.csv')
df_test.head()

,Question,Answer
0,What are some of the biggest challenges you fa...,Making friends from different culture is diffi...
1,How do you make friends and build a social lif...,"I did part-time jobs at a school center, and a..."
2,What are some underrated travel destinations t...,Some parks in Bloomington are good enough for ...
3,How do you manage travel expenses while studyi...,I traveled out of state not more than 2 times ...
4,What is one place in the world that you���ve a...,I really want to visit Mexico after I watched ...


In [36]:
df_test["Question"][0]

'What are some of the biggest challenges you face when adjusting to a new culture?'

In [39]:
def zero_shot(question, knowledge):
    prompt = f"""You are Yishan. Here are some of Yishan's past answers:
    {knowledge}
    Based on these answers, respond in Yishan's style.
    Question:{question}
    Answer:"""
    
    
    response = client.responses.create(
        model = 'gpt-4o',
        input=[
            {
                "role":"user",
                "content":prompt
            }]

    )
    return response.output_text

In [40]:
def chain_of_thought(question, knowledge):
    prompt = f"""You are Yishan. Think step by step before answering.
    Here are some of Yishan's past answers:
    {knowledge}
    Based on these answers, respond in Yishan's style

    Question: {question}
    Answer:"""

    response = client.responses.create(
        model = 'gpt-4o',
        input=[
            {
                "role":"user",
                "content":prompt
            }]

    )
    return response.output_text
  

In [42]:
question = df_test["Question"][0]
chain_of_thought_response = chain_of_thought(question, yishan_knowledge)
zero_shot_response = zero_shot(question, yishan_knowledge)

In [43]:
print(chain_of_thought_response)
print(zero_shot_response)

When adjusting to a new culture, I find that the biggest challenges are understanding and adapting to social norms and communication styles. Every culture has its unique way of interacting, whether it's how people greet each other or express their emotions. It can be tricky to navigate these differences and ensure I don't unintentionally offend anyone. Additionally, language barriers can sometimes add to the complexity, making it harder to connect deeply with others. Lastly, missing familiar routines and comforts from home can be tough, but I try to embrace the adventure and learn as much as I can.
Adjusting to a new culture can be quite challenging. One of the biggest challenges I face is dealing with cultural misunderstandings. Each culture has its own set of norms and values, and it takes time to learn and adapt to them. Language barriers can also make communication tricky, as nuances and expressions may not translate perfectly. Additionally, adjusting to different social dynamics a

In [46]:
# get cosine similarity

import math
import re
from collections import Counter

WORD = re.compile(r"\w+")


def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

In [47]:
def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)

In [50]:
answer = df_test['Answer'][0]
answer

"Making friends from different culture is difficult for me. I'm worried about mentioning something forbidden or controversial in their religion or politics. "

In [51]:
vector1 = text_to_vector(zero_shot_response)
vector2 = text_to_vector(answer)
cosine = get_cosine(vector1, vector2)
print('Zero shot cosine:', cosine)

Zero shot cosine: 0.09230769230769231


In [52]:
vector1 = text_to_vector(chain_of_thought_response)
vector2 = text_to_vector(answer)
cosine = get_cosine(vector1, vector2)
print('Chain of thoughts cosine:', cosine)

Chain of thoughts cosine: 0.14586499149789456
